In [1]:
import import_ipynb
from model import generator, discriminator
from config import(
    EPOCHS_LR,
    BATCH_SIZE,
    NOISE_LENGTH,
    DEVICE,
    BASE_OUTPUT,
)
from dataset import train_loader, val_loader
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from torch import optim, nn
import matplotlib.pyplot as plt

importing Jupyter notebook from model.ipynb
importing Jupyter notebook from config.ipynb
importing Jupyter notebook from dataset.ipynb


In [2]:
def save_loss_plot(out_dir, train_loss, val_loss , x):
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()
    train_ax.plot(train_loss, color='tab:blue')
    train_ax.set_xlabel('iterations')
    train_ax.set_ylabel('train loss')
    valid_ax.plot(val_loss, color='tab:red')
    valid_ax.set_xlabel('iterations')
    valid_ax.set_ylabel('validation loss')
    figure_1.savefig(f"{out_dir}/{x}train_loss.png")
    figure_2.savefig(f"{out_dir}/{x}valid_loss.png")
    print('SAVING PLOTS COMPLETE...')
    
    plt.close('all')

In [3]:
H = {"G_train_loss": [], "G_valid_loss": [], "D_train_loss": [], "D_valid_loss": []}

In [4]:
G = generator(128).to(DEVICE)
D = discriminator(128).to(DEVICE)

In [5]:
G.weight_init(mean=0.0, std=0.02)
D.weight_init(mean=0.0, std=0.02)

BCE_loss = nn.BCELoss()

In [6]:
for n, (epochs, lr) in enumerate(EPOCHS_LR):
    G_opt = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
    D_opt = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
    for epoch in range(epochs):
        D.train()
        G.train()
        D_train_loss = 0.0
        G_train_loss = 0.0
        for i, (img, _) in tqdm(enumerate(train_loader), total=len(train_loader)):
            # Train D
            D_opt.zero_grad()
            mini_batch = img.size()[0]
            img = img.to(DEVICE)    
            
            # 真圖片輸入 D
            D_result = D(img).squeeze()
            # 計算真圖片的loss
            D_real_loss = BCE_loss(D_result, torch.ones_like(D_result))
            # 新建一個隨機變數
            noise = (
                torch.randn((mini_batch, NOISE_LENGTH)).
                view((-1, NOISE_LENGTH, 1, 1)).
                to(DEVICE)
            )
            # 生假圖片
            img_fake = G(noise)
            # 將假圖片輸入至判別器D
            D_result = D(img_fake).squeeze()
            # 計算假圖片的loss
            D_fake_loss = BCE_loss(D_result, torch.ones_like(D_result))
            # 真假圖片的判別器loss相加之和反向傳播
            D_train_loss = D_real_loss + D_fake_loss
            D_train_loss.backward()
            D_opt.step()
            
            # Train G
            G_opt.zero_grad()
            noise = (
                torch.randn((mini_batch, NOISE_LENGTH)).
                view((-1, NOISE_LENGTH, 1, 1)).
                to(DEVICE)
            )
            # 生成假樸片
            img_fake = G(noise)
            # 輸入判別器D計算loss
            D_result = D(img_fake).squeeze()
            # 給假圖片打上真標籤，計算loss
            G_train_loss = BCE_loss(D_result, torch.ones_like(D_result))
            # 生成器反向傳播
            G_train_loss.backward()
            G_opt.step()
        H["D_train_loss"].append(D_train_loss)
        H["G_train_loss"].append(G_train_loss)
        print('D train loss : {} , G train loss : {}'.format( D_train_loss, G_train_loss))
        
        D_valid_loss = 0.0
        G_valid_loss = 0.0
        with torch.no_grad():
            D.eval()
            G.eval()
            for i, (img,_) in tqdm(enumerate(val_loader), total=len(val_loader)):
                mini_batch = img.size()[0]
                img = img.to(DEVICE)
                # 真圖片輸入 D
                D_result = D(img).squeeze()
                # 計算真圖片的loss
                D_real_loss = BCE_loss(D_result, torch.ones_like(D_result))
                # 新建一個隨機變數
                noise = (
                    torch.randn((mini_batch, NOISE_LENGTH)).
                    view((-1, NOISE_LENGTH, 1, 1)).
                    to(DEVICE)
                )
                # 生假圖片
                img_fake = G(noise)
                # 將假圖片輸入至判別器D
                D_result = D(img_fake).squeeze()
                # 計算假圖片的loss
                D_fake_loss = BCE_loss(D_result, torch.ones_like(D_result))
                # 真假圖片的判別器loss相加之和反向傳播
                D_val_loss = D_real_loss + D_fake_loss

                noise = (
                    torch.randn((mini_batch, NOISE_LENGTH)).
                    view((-1, NOISE_LENGTH, 1, 1)).
                    to(DEVICE)
                )
                # 生成假樸片
                img_fake = G(noise)
                # 輸入判別器D計算loss
                D_result = D(img_fake).squeeze()
                # 給假圖片打上真標籤，計算loss
                G_val_loss = BCE_loss(D_result, torch.ones_like(D_result))
            H["D_valid_loss"].append(D_val_loss)
            H["G_valid_loss"].append(G_val_loss)
            print('D val loss : {} , G val loss : {}'.format( D_val_loss, G_val_loss))
        save_loss_plot(
            BASE_OUTPUT, H["G_train_loss"],  H["G_valid_loss"], 'G'
        )
        save_loss_plot(
            BASE_OUTPUT, H["D_train_loss"],  H["D_valid_loss"], 'D'
        )
        torch.save(G.state_dict, f'{BASE_OUTPUT}/Gmodel.pth')
        torch.save(D.state_dict, f'{BASE_OUTPUT}/Dmodel.pth')

100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [00:59<00:00,  1.40it/s]


D train loss : 0.9031909108161926 , G train loss : 1.6490641883137869e-06


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:07<00:00,  1.46it/s]


D val loss : 0.2241174727678299 , G val loss : 0.0010753822280094028
SAVING PLOTS COMPLETE...
SAVING PLOTS COMPLETE...


100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [00:57<00:00,  1.44it/s]


...

D val loss : 0.2525908648967743 , G val loss : 8.675335993757471e-05
SAVING PLOTS COMPLETE...
SAVING PLOTS COMPLETE...


100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [00:51<00:00,  1.61it/s]


D train loss : 0.9014678001403809 , G train loss : 0.0


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:06<00:00,  1.65it/s]


D val loss : 0.23843835294246674 , G val loss : 8.589559729443863e-05
SAVING PLOTS COMPLETE...
SAVING PLOTS COMPLETE...
